In [ ]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from datetime import datetime

import operator
import time
import os

In [ ]:
font_name = "Songti SC"
matplotlib.rcParams['font.family']=font_name
matplotlib.rcParams['axes.unicode_minus']=False
matplotlib.rcParams['axes.labelweight'] = 'bold'

In [ ]:
conn = sqlite3.connect("./workdir/entities.sqlite3")
stat = conn.execute("""
    WITH th_video AS (
        SELECT 
            auto_certified_video.aid, 
            datetime(publish_time, 'unixepoch') AS publish_time,
            ownership
        FROM auto_certified_video
        LEFT JOIN video ON auto_certified_video.aid = video.aid
    )
    SELECT 
        strftime('%Y-%m', publish_time) AS "month", 
        count(aid) AS video_count, 
        max(aid) AS max_aid,
        sum(ownership = 1) AS original_count
    FROM th_video
    GROUP BY "month"
    HAVING "month" IS NOT NULL
    ORDER BY "month" ASC
    """).fetchall()
conn.close()
dates = []
counts = []
delta_aids = [0]
ratios = []
original_counts = []
for i, x in enumerate(stat):
    if i == len(stat) - 1:
        break
    dates.append(datetime.strptime(x[0], '%Y-%m'))
    count = x[1]
    counts.append(count)
    delta_aid = x[2] - delta_aids[-1]
    delta_aids.append(delta_aid)
    ratios.append(count / delta_aid)
    original_counts.append(x[3])
delta_aids.pop(0)

In [ ]:
fig, ax1 = plt.subplots()
plt.title('bilibili上东方相关视频的简单统计')

ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax1.xaxis.set_minor_locator(mdates.MonthLocator())
ax1.set_xlabel('时间')
ax1.grid(True, which='major', axis='x', alpha=1/3)

ax2 = ax1.twinx()

ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax2.xaxis.set_minor_locator(mdates.MonthLocator())
ax2.set_xlabel('时间')
ax2.grid(True, which='major', axis='x', alpha=1/3)

color='tab:blue'
ax1.set_ylabel('当月新增稿件占总新增稿件比例', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

ax1.plot(dates, ratios, color=color)
ax1.fill_between(dates, ratios, alpha=0.4, color=color)

color='tab:green'
ax2.plot(dates, counts, color=color)
ax2.set_ylabel('当月新增稿件数 (有遗漏)', color=color)
ax2.tick_params(axis='y', labelcolor=color)

ax2.plot(dates, counts, label="总数", color=color)
ax2.plot(dates, original_counts, label="自制稿件数", linestyle='dashed', alpha=0.75, color=color)
ax2.plot(dates, list(map(operator.sub, counts, original_counts)), label="转载稿件数", linestyle='dotted', alpha=0.75, color=color)

fig.autofmt_xdate()
fig.tight_layout()

plt.legend(loc="upper right", bbox_to_anchor=(0.9, 1))
#plt.show()

plt.savefig("{}/{}".format(os.path.expanduser("~/Desktop"), 'simple-stats.png'), dpi=240)